In [ ]:
import logging
from datetime import datetime

from _correlation import cluster_correlation_search
import clustering
from common import *
import networkx as nx
import numpy as np

In [ ]:
def get_clusters(adj_matrix, hyperparameters, seed=0):
    G = clustering._adjacency_matrix_to_nxgraph(adj_matrix)
    
    clusters = clustering.correlation_clustering(G, **hyperparameters)
    return clustering._convert_graph_cluster_list_set_to_list(G, clusters)

In [ ]:
score_paths = {
    'rusemshift-finetune': u'Data/l1ndotn_schemas/rusemshift/finetune/german/{0}/dev.*.scores',
    'rusemshift-train': u'Data/l1ndotn_schemas/rusemshift/train/german/{0}/dev.*.scores',
    'ru-ru': u'Data/l1ndotn_schemas/ru-ru/german/{0}/dev.*.scores',
    'en-en': u'Data/l1ndotn_schemas/en-en/german/{0}/dev.*.scores'
}

In [ ]:
method = "correlation_clustering"
no_experiments=5

In [ ]:
logging.basicConfig(
    filename=f"../logs/{method}/logs.txt",
    filemode="a+",
    format="%(asctime)s : %(message)s",
    datefmt="%y-%m-%d %I:%M:%S %p",
    level=logging.INFO
)

In [ ]:
def get_thresholds_for_cc(get_data, score_paths):
    model_hyperparameter_combinations = []
    ans = {}

    senses, scores = get_data(score_paths)
    scalers = get_scaling(scores)
    
    for score_path in score_paths:
        for sense in [10]:
            for attempt in [2000]:
                for iteration in [50000]:
                    model_hyperparameter_combinations.append(
                        {
                            "threshold_cc": scalers[score_path].transform(np.array(0.5).reshape(1, -1)).item(0),
                            "max_attempts": attempt,
                            "max_iters": iteration,
                            "max_senses": sense
                        }
                    )

        ans[score_path] = model_hyperparameter_combinations
        model_hyperparameter_combinations = []

    return ans


In [ ]:
start_time = datetime.now()

model_hyperparameter_combinations = get_thresholds_for_cc(
    get_dwug_data_annotated_only, score_paths
)

grid_search_without_nclusters(
    get_dwug_data_annotated_only,
    get_clusters,
    score_paths,
    model_hyperparameter_combinations,
    include_binarize=True,
    method=f"{method}",
    logger_message={"logging": logging},
    cache={
        "name": "../outputs/experiment-results/{result}/{method}/cache/cache_dwug_data_annotated_only.csv",
        "result_name": "../outputs/experiment-results/{result}/{method}/results/results_dwug_data_annotated_only.csv",
    },
    run_experiments=no_experiments,
    dataset="dwug_data_annotated_only"
)

print(f"Elapsed time: {datetime.now() - start_time}")

In [ ]:
start_time = datetime.now()

model_hyperparameter_combinations = get_thresholds_for_cc(
    get_dwug_old_data_annotated_only, score_paths
)

grid_search_without_nclusters(
    get_dwug_old_data_annotated_only,
    get_clusters,
    score_paths,
    model_hyperparameter_combinations,
    include_binarize=True,
    method=f"{method}",
    logger_message={"logging": logging},
    cache={
        "name": "../outputs/experiment-results/{result}/{method}/cache/cache_dwug_old_data_annotated_only.csv",
        "result_name": "../outputs/experiment-results/{result}/{method}/results/results_dwug_old_data_annotated_only.csv",
    },
    run_experiments=no_experiments,
    dataset="dwug_old_data_annotated_only",
)

print(f"Elapsed time: {datetime.now() - start_time}")

In [ ]:
start_time = datetime.now()

model_hyperparameter_combinations = get_thresholds_for_cc(
    get_dwug_new_data_annotated_only, score_paths
)

grid_search_without_nclusters(
    get_dwug_new_data_annotated_only,
    get_clusters,
    score_paths,
    model_hyperparameter_combinations,
    include_binarize=True,
    method=f"{method}",
    logger_message={"logging": logging},
    cache={
        "name": "../outputs/experiment-results/{result}/{method}/cache/cache_dwug_new_data_annotated_only.csv",
        "result_name": "../outputs/experiment-results/{result}/{method}/results/results_dwug_new_data_annotated_only.csv",
    },
    run_experiments=no_experiments,
    dataset="dwug_new_data_annotated_only",
)

print(f"Elapsed time: {datetime.now() - start_time}")